In [0]:
%python
df=spark.read.table("dev.richa_bronze.sales")

In [0]:
%python
df_cleaned=df.dropDuplicates().dropna().drop("ingestion_date")

In [0]:
%python
df_cleaned.write.mode("overwrite").saveAsTable("dev.richa_silver.sales_cleaned")

In [0]:
select * from dev.richa_bronze.products

In [0]:
create or replace table dev.richa_silver.product_scd (product_id int, product_name string, product_category string, product_price double)

In [0]:
WITH deduplicated_source AS (
    SELECT 
        product_id, 
        product_name, 
        product_category, 
        product_price,
        ROW_NUMBER() OVER (PARTITION BY product_id ORDER BY seqNum DESC) as row_num
    FROM dev.richa_bronze.products
)
MERGE INTO dev.richa_silver.product_scd t
USING (SELECT * FROM deduplicated_source WHERE row_num = 1) s
ON t.product_id = s.product_id
WHEN MATCHED THEN 
    UPDATE SET 
        t.product_name = s.product_name,
        t.product_category = s.product_category,
        t.product_price = s.product_price
WHEN NOT MATCHED THEN 
    INSERT (product_id, product_name, product_category, product_price) 
    VALUES (s.product_id, s.product_name, s.product_category, s.product_price)

In [0]:

MERGE INTO dev.richa_silver.product_scd t
USING dev.richa_bronze.products s
ON t.product_id = s.product_id
when matched and operation= "DELETE" then delete
when matched then update set 
t.product_name = s.product_name,
t.product_category = s.product_category,
t.product_price = s.product_price
when not matched then insert 
(product_id, product_name, product_category,product_price) 
values 
(s.product_id, s.product_name, s.product_category,s.product_price)


In [0]:
delete from dev.richa_silver.product_scd